In [1]:
## RAG Application With GPT-4o

In [2]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

# 1. Get a Data Loader


In [3]:
from langchain_community.document_loaders import WebBaseLoader


In [4]:
! pip install bs4

In [5]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
data = loader.load()
data

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly un

# 2. Convert data to Vector Database


In [7]:
! pip install langchain_objectbox

  Using cached langchain_objectbox-0.1.0a4-py3-none-any.whl.metadata (4.5 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached objectbox-4.0.0a1-py3-none-any.whl.metadata (6.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
Using cached langchain_objectbox-0.1.0a4-py3-none-any.whl (6.6 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
Using cached objectbox-4.0.0a1-py3-none-any.whl (4.0 MB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.39
    Uninstalling langchain-core-0.1.39:
      Successfully uninstalled langchain-core-0.1.39


In [9]:
! pip install langchain_openai

  Using cached langchain_openai-0.1.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached openai-1.30.1-py3-none-any.whl.metadata (21 kB)
  Using cached tiktoken-0.7.0-cp39-cp39-win_amd64.whl.metadata (6.8 kB)
  Using cached regex-2024.5.15-cp39-cp39-win_amd64.whl.metadata (41 kB)
Using cached langchain_openai-0.1.7-py3-none-any.whl (34 kB)
Using cached openai-1.30.1-py3-none-any.whl (320 kB)
Using cached tiktoken-0.7.0-cp39-cp39-win_amd64.whl (798 kB)
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   ------------------------------ --------- 204.8/269.0 kB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 269.0/269.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [11]:

from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings


ImportError: cannot import name 'InvalidToolCall' from 'langchain_core.messages' (c:\Users\DELL\Desktop\jupyter\langchain\venv\lib\site-packages\langchain_core\messages\__init__.py)

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [13]:
documents

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform

In [14]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

ImportError: cannot import name 'InvalidToolCall' from 'langchain_core.messages' (c:\Users\DELL\Desktop\jupyter\langchain\venv\lib\site-packages\langchain_core\messages\__init__.py)

# 3. Make a RAG pipeline


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [ ]:
llm = ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
prompt = hub.pull("rlm/rag-prompt")
prompt



In [ ]:
prompt

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [ ]:
question = "Explain what is langsmith"
result = qa_chain({"query": question })
result

In [ ]:
result["result"]

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["result"])

In [ ]:
question = "Explain Monitoring and A/B Testing in langsmith"
result = qa_chain({"query": question })
result